In [ ]:
# Define token types
INTEGER = 'INTEGER'
BOOLEAN = 'BOOLEAN'
PLUS = 'PLUS'
MINUS = 'MINUS'
MULTIPLY = 'MULTIPLY'
DIVIDE = 'DIVIDE'
EQUALS = 'EQUALS'
IF = 'IF'
ELSE = 'ELSE'
PRINT = 'PRINT'
IDENTIFIER = 'IDENTIFIER'
EOF = 'EOF'

# Define a Token class
class Token:
    def __init__(self, type, value):
        self.type = type
        self.value = value

# Define a Lexer to tokenize input
class Lexer:
    def __init__(self, text):
        self.text = text
        self.pos = 0
        self.current_char = self.text[self.pos]

    def error(self):
        raise Exception('Invalid character')

    def advance(self):
        self.pos += 1
        if self.pos < len(self.text):
            self.current_char = self.text[self.pos]
        else:
            self.current_char = None

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):
        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)

    def get_next_token(self):
      while self.current_char is not None:
          if self.current_char.isspace():
              self.skip_whitespace()
              continue

          if self.current_char.isdigit():
              return Token(INTEGER, self.integer())

          if self.current_char == '+':
              self.advance()
              return Token(PLUS, '+')

          if self.current_char == '-':
              self.advance()
              return Token(MINUS, '-')

          if self.current_char == '*':
              self.advance()
              return Token(MULTIPLY, '*')

          if self.current_char == '/':
              self.advance()
              return Token(DIVIDE, '/')

          if self.current_char == '=':
              self.advance()
              return Token(EQUALS, '=')

          if self.current_char == '(':
              self.advance()
              return Token('LPAREN', '(')

          if self.current_char == ')':
              self.advance()
              return Token('RPAREN', ')')

          if self.current_char == '{':
              self.advance()
              return Token('LBRACE', '{')

          if self.current_char == '}':
              self.advance()
              return Token('RBRACE', '}')

          if self.current_char == ';':
              self.advance()
              return Token('SEMI', ';')

          if self.current_char == '"':
              self.advance()
              return Token('QUOTE', '"')

          if self.current_char.isalpha():
              return self.keyword_or_identifier()

          self.error()

      return Token(EOF, None)

    def keyword_or_identifier(self):
      result = ''
      while self.current_char is not None and (self.current_char.isalnum() or self.current_char == '_'):
          result += self.current_char
          self.advance()

      if result == 'if':
          return Token(IF, 'if')
      elif result == 'else':
          return Token(ELSE, 'else')
      elif result == 'print':
          return Token(PRINT, 'print')
      elif result == 'true':
          return Token(BOOLEAN, True)  # Recognize 'true' as boolean True
      elif result == 'false':
          return Token(BOOLEAN, False)  # Recognize 'false' as boolean False
      else:
          return Token(IDENTIFIER, result)

# Define the Parser
class Parser:
    def __init__(self, lexer):
        self.lexer = lexer
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
      token = self.current_token
      if token.type == INTEGER:
          self.eat(INTEGER)
          return token.value
      elif token.type == BOOLEAN:
          self.eat(BOOLEAN)
          return int(token.value)  # Convert boolean to integer
      elif token.type == IDENTIFIER:
          # Assuming identifiers represent variables holding either boolean or integer values
          variable_name = token.value
          self.eat(IDENTIFIER)
          return self.variables[variable_name]
      elif token.type == 'LPAREN':
          self.eat('LPAREN')
          result = self.expr()
          self.eat('RPAREN')
          return result


    def term(self):
        result = self.factor()

        while self.current_token.type in (MULTIPLY, DIVIDE):
            token = self.current_token
            if token.type == MULTIPLY:
                self.eat(MULTIPLY)
                result *= self.factor()
            elif token.type == DIVIDE:
                self.eat(DIVIDE)
                result /= self.factor()

        return result

    def expr(self):
      result = self.term()

      while self.current_token.type in (PLUS, MINUS):
          token = self.current_token
          if token.type == PLUS:
              self.eat(PLUS)
              result += self.term()
          elif token.type == MINUS:
              self.eat(MINUS)
              result -= self.term()

      return result


    def parse(self):
        return self.expr()

# Test the parser
def main():
    while True:
        try:
            text = input('MiniLang > ')
        except EOFError:
            break
        if not text:
            continue
        lexer = Lexer(text)
        parser = Parser(lexer)
        result = parser.parse()
        print(result)

if __name__ == '__main__':
    main()


MiniLang > 10 - 4 / 2;
8.0
